<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#模型调优-(判定贷款用户是否逾期)" data-toc-modified-id="模型调优-(判定贷款用户是否逾期)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>模型调优 (判定贷款用户是否逾期)</a></span><ul class="toc-item"><li><span><a href="#数据集划分" data-toc-modified-id="数据集划分-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>数据集划分</a></span></li><li><span><a href="#模型评估" data-toc-modified-id="模型评估-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>模型评估</a></span></li><li><span><a href="#LR模型" data-toc-modified-id="LR模型-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>LR模型</a></span></li><li><span><a href="#SVM模型" data-toc-modified-id="SVM模型-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>SVM模型</a></span></li><li><span><a href="#决策树模型" data-toc-modified-id="决策树模型-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>决策树模型</a></span></li><li><span><a href="#XGBoost模型" data-toc-modified-id="XGBoost模型-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>XGBoost模型</a></span></li><li><span><a href="#LightGBM模型" data-toc-modified-id="LightGBM模型-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>LightGBM模型</a></span></li></ul></li><li><span><a href="#遇到的问题" data-toc-modified-id="遇到的问题-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>遇到的问题</a></span></li><li><span><a href="#Reference" data-toc-modified-id="Reference-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Reference</a></span></li></ul></div>

# 模型调优 (判定贷款用户是否逾期)

给定金融数据，预测贷款用户是否会逾期。
（status是标签：0表示未逾期，1表示逾期。）

**Task6（模型调优）** - 使用网格搜索对模型进行调优, 并采用五折交叉验证的方式进行模型评估

In [1]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# 数据集预览
data = pd.read_csv('data.csv')
# 去重
data.drop_duplicates(inplace=True)
print(data.shape)
data.head()

(4754, 90)


,Unnamed: 0,custid,trade_no,bank_card_no,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,5,2791858,20180507115231274000000023057383,卡号1,0.01,0.99,0,0.90,0.55,0.313,...,2900.0,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0
1,10,534047,20180507121002192000000023073000,卡号1,0.02,0.94,2000,1.28,1.00,0.458,...,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0
2,12,2849787,20180507125159718000000023114911,卡号1,0.04,0.96,0,1.00,1.00,0.114,...,1600.0,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0
3,13,1809708,20180507121358683000000388283484,卡号1,0.00,0.96,2000,0.13,0.57,0.777,...,3200.0,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0
4,14,2499829,20180507115448545000000388205844,卡号1,0.01,0.99,0,0.46,1.00,0.175,...,2300.0,1630.0,8300.0,79.0,2.0,2.0,8400.0,8250.0,22.0,120.0


In [2]:
import pickle
# 载入特征
with open('feature.pkl', 'rb') as f:
    X = pickle.load(f)

# 观测正负样本是否均衡
y = data.status
y.value_counts()

0    3561
1    1193
Name: status, dtype: int64

## 数据集划分

In [3]:
# 划分训练集测试集
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=2333)

# 特征归一化
std = StandardScaler()
X_train = std.fit_transform(X_train)
X_test = std.transform(X_test)

## 模型评估

In [4]:
from sklearn.metrics import accuracy_score, roc_auc_score

def model_metrics(clf, X_train, X_test, y_train, y_test):
    # 预测
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)
    
    y_train_proba = clf.predict_proba(X_train)[:,1]
    y_test_proba = clf.predict_proba(X_test)[:,1]
    
    # 准确率
    print('[准确率]', end = ' ')
    print('训练集：', '%.4f'%accuracy_score(y_train, y_train_pred), end = ' ')
    print('测试集：', '%.4f'%accuracy_score(y_test, y_test_pred))
    
    # auc取值：用roc_auc_score或auc
    print('[auc值]', end = ' ')
    print('训练集：', '%.4f'%roc_auc_score(y_train, y_train_proba), end = ' ')
    print('测试集：', '%.4f'%roc_auc_score(y_test, y_test_proba))

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from xgboost.sklearn import XGBClassifier
from lightgbm.sklearn import LGBMClassifier

## LR模型

In [6]:
lr = LogisticRegression()
param = {'C': [1e-3,0.01,0.1,1,10,100,1e3], 'penalty':['l1', 'l2']}

gsearch = GridSearchCV(lr, param_grid = param,scoring ='roc_auc', cv=5)
gsearch.fit(X_train, y_train)

print('最佳参数：',gsearch.best_params_)
print('训练集的最佳分数：', gsearch.best_score_)
print('测试集的最佳分数：', gsearch.score(X_test, y_test))

最佳参数： {'C': 0.1, 'penalty': 'l1'}
训练集的最佳分数： 0.7848352043059409
测试集的最佳分数： 0.8002199819583141


In [7]:
lr = LogisticRegression(C = 0.1, penalty = 'l1')
lr.fit(X_train, y_train)
model_metrics(lr, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.7962 测试集： 0.8080
[auc值] 训练集： 0.8008 测试集： 0.8002


## SVM模型

> 调参范围可设为'gamma':[0.001,0.01,0.1,1,10,100], 'C':[0.001,0.01,0.1,1,10,100]}。鉴于时间原因, 下面网格搜索时选用较小区间。

In [6]:
# 线性SVM
svm_linear = svm.SVC(kernel = 'linear', probability=True)
param = {'C':[0.01,0.1,1]}
gsearch = GridSearchCV(svm_linear, param_grid = param,scoring ='roc_auc', cv=5)
gsearch.fit(X_train, y_train)

print('最佳参数：',gsearch.best_params_)
print('训练集的最佳分数：', gsearch.best_score_)
print('测试集的最佳分数：', gsearch.score(X_test, y_test))

最佳参数： {'C': 0.01}
训练集的最佳分数： 0.7873824454604643
测试集的最佳分数： 0.8114643834966053


In [7]:
svm_linear = svm.SVC(C = 0.01, kernel = 'linear', probability=True)
svm_linear.fit(X_train, y_train)
model_metrics(svm_linear, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.7839 测试集： 0.7905
[auc值] 训练集： 0.8036 测试集： 0.8115


In [8]:
# 多项式SVM
svm_poly = svm.SVC(kernel = 'poly', probability=True)
param = {'C':[0.01,0.1,1]}
gsearch = GridSearchCV(svm_poly, param_grid = param,scoring ='roc_auc', cv=5)
gsearch.fit(X_train, y_train)

print('最佳参数：',gsearch.best_params_)
print('训练集的最佳分数：', gsearch.best_score_)
print('测试集的最佳分数：', gsearch.score(X_test, y_test))

最佳参数： {'C': 0.01}
训练集的最佳分数： 0.7376911042619483
测试集的最佳分数： 0.7289131203148326


In [9]:
svm_poly =  svm.SVC(C = 0.01, kernel = 'poly', probability=True)
svm_poly.fit(X_train, y_train)
model_metrics(svm_poly, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.7541 测试集： 0.7568
[auc值] 训练集： 0.8526 测试集： 0.7289


In [10]:
# 高斯SVM
svm_rbf = svm.SVC(probability=True)
param = {'gamma':[0.01,0.1,1,10], 
         'C':[0.01,0.1,1]}
gsearch = GridSearchCV(svm_poly, param_grid = param,scoring ='roc_auc', cv=5)
gsearch.fit(X_train, y_train)

print('最佳参数：',gsearch.best_params_)
print('训练集的最佳分数：', gsearch.best_score_)
print('测试集的最佳分数：', gsearch.score(X_test, y_test))

最佳参数： {'C': 0.01, 'gamma': 0.01}
训练集的最佳分数： 0.7395294751089104
测试集的最佳分数： 0.7244290756009939


In [12]:
svm_rbf =  svm.SVC(gamma = 0.01, C =0.01 , probability=True)
svm_rbf.fit(X_train, y_train)
model_metrics(svm_rbf, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.7475 测试集： 0.7526
[auc值] 训练集： 0.8448 测试集： 0.7980


In [16]:
# sigmoid - SVM
svm_sigmoid = svm.SVC(kernel = 'sigmoid',probability=True)
param = {'C':[0.01,0.1,1]}
gsearch = GridSearchCV(svm_sigmoid, param_grid = param,scoring ='roc_auc', cv=5)
gsearch.fit(X_train, y_train)

print('最佳参数：',gsearch.best_params_)
print('训练集的最佳分数：', gsearch.best_score_)
print('测试集的最佳分数：', gsearch.score(X_test, y_test))

最佳参数： {'C': 0.01}
训练集的最佳分数： 0.771523961695314
测试集的最佳分数： 0.8009427044592505


In [17]:
svm_sigmoid =  svm.SVC(C = 0.01, kernel = 'sigmoid',probability=True)
svm_sigmoid.fit(X_train, y_train)
model_metrics(svm_sigmoid, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.7475 测试集： 0.7526
[auc值] 训练集： 0.7747 测试集： 0.8009


## 决策树模型

1）首先对决策树最大深度max_depth和内部节点再划分所需最小样本数min_samples_split进行网格搜索。

In [8]:
param = {'max_depth':range(3,14,2), 'min_samples_split':range(100,801,200)}
gsearch = GridSearchCV(DecisionTreeClassifier(max_depth=8,min_samples_split=300,min_samples_leaf=20, max_features='sqrt'),
                       param_grid = param,scoring ='roc_auc', cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'max_depth': 9, 'min_samples_split': 100}, 0.7082443260701551)

2）对内部节点再划分所需最小样本数min_samples_split和叶子节点最少样本数min_samples_leaf一起调参。

In [10]:
param = {'min_samples_split':range(50,1000,100), 'min_samples_leaf':range(60,101,10)}
gsearch = GridSearchCV(DecisionTreeClassifier(max_depth=9,min_samples_split=100,min_samples_leaf=20, max_features='sqrt'),
                       param_grid = param,scoring ='roc_auc', cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'min_samples_leaf': 90, 'min_samples_split': 350}, 0.7084690535238208)

3）再对最大特征数max_features进行网格搜索

In [11]:
param = {'max_features':range(7,20,2)}
gsearch = GridSearchCV(DecisionTreeClassifier(max_depth=9,min_samples_split=350,min_samples_leaf=90, max_features='sqrt'),
                       param_grid = param,scoring ='roc_auc', cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'max_features': 17}, 0.7178495862278447)

In [13]:
dt = DecisionTreeClassifier(max_depth=9,min_samples_split=350,min_samples_leaf=90,max_features=17)
dt.fit(X_train, y_train)
model_metrics(dt, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.7622 测试集： 0.7652
[auc值] 训练集： 0.7413 测试集： 0.6948


## XGBoost模型

1、max_depth = 5 :这个参数的取值最好在3-10之间。我选的起始值为5，但是你也可以选择其它的值。起始值在4-6之间都是不错的选择。

2、min_child_weight = 1:在这里选了一个比较小的值，因为这是一个极不平衡的分类问题。因此，某些叶子节点下的值会比较小。

3、gamma = 0: 起始值也可以选其它比较小的值，在0.1到0.2之间就可以。这个参数后继也是要调整的。

4、subsample, colsample_bytree = 0.8: 这个是最常见的初始值了。典型值的范围在0.5-0.9之间。

首先看一下默认参数的结果

In [6]:
import warnings
warnings.filterwarnings("ignore")

xgb0 = XGBClassifier()
xgb0.fit(X_train, y_train)

model_metrics(xgb0, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.8560 测试集： 0.8066
[auc值] 训练集： 0.9154 测试集： 0.7926


1) 首先从步长(learning rate)和迭代次数(n_estimators)入手。

开始选择一个较小的步长来网格搜索最好的迭代次数。这里，我们将步长初始值设置为0.1, 对于迭代次数进行网格搜索。

> 在学习速率为0.1时，理想的决策树数目是40。

In [10]:
param_test = {'n_estimators':range(20,200,20)}
gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=140, max_depth=5, 
                                                  min_child_weight=1, gamma=0, subsample=0.8, 
                                                  colsample_bytree=0.8, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'n_estimators': 40}, 0.7796727871532119)

2) max_depth 和 min_child_weight 参数调优

> 取'max_depth': 3, 'min_child_weight': 10

In [13]:
param_test = {'max_depth':range(3,10,2), 'min_child_weight':range(1,12,2)}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=40, max_depth=5, 
                                                  min_child_weight=1, gamma=0, subsample=0.8, 
                                                  colsample_bytree=0.8, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'max_depth': 3, 'min_child_weight': 11}, 0.7906323843808692)

可以看出理想的max_depth值为3，理想的min_child_weight值为11。在这个值附近我们可以再进一步调整，来找出理想值。

In [14]:
param_test = {'max_depth':[3,4,5], 'min_child_weight':[10,11,12]}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=40, max_depth=5, 
                                                  min_child_weight=1, gamma=0, subsample=0.8, 
                                                  colsample_bytree=0.8, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'max_depth': 3, 'min_child_weight': 10}, 0.7910360710832215)

3) gamma参数调优

> 取gamma = 0.5

In [15]:
param_test = {'gamma':[i/10 for i in range(1,6)]}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=40, max_depth=3, 
                                                  min_child_weight=10, gamma=0, subsample=0.8, 
                                                  colsample_bytree=0.8, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'gamma': 0.5}, 0.7910384519778837)

4）调整subsample 和 colsample_bytree 参数

In [17]:
param_test = {'subsample':[i/10 for i in range(5,10)], 'colsample_bytree':[i/10 for i in range(5,10)]}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=40, max_depth=3, 
                                                  min_child_weight=10, gamma=0.5, subsample=0.8, 
                                                  colsample_bytree=0.8, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'colsample_bytree': 0.7, 'subsample': 0.6}, 0.7925678713705064)

从这里可以看出来，subsample理想取值0.6, colsample_bytree理想取值都是0.7。现在，我们以0.05为步长，在这个值附近尝试取值。

In [20]:
param_test = { 'subsample':[i/100 for i in range(55,66,5)], 'colsample_bytree':[i/100 for i in range(65,76,5)]}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=40, max_depth=3, 
                                                  min_child_weight=10, gamma=0.5, subsample=0.8, 
                                                  colsample_bytree=0.8, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'colsample_bytree': 0.75, 'subsample': 0.6}, 0.7941078436578545)

5）正则化参数调优

> #'reg_lambda': [0.2, 0.4, 0.6, 0.8, 1]

In [21]:
#  'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
param_test = {'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]}

gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=40, max_depth=3, 
                                                  min_child_weight=10, gamma=0.5, subsample=0.6, 
                                                  colsample_bytree=0.75, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'reg_alpha': 0.1}, 0.7947425180264991)

6）回到第1）步，降低学习速率, 调整迭代次数

In [23]:
param_test = {'n_estimators':range(20,200,20)}
gsearch = GridSearchCV(estimator = XGBClassifier(learning_rate =0.01, n_estimators=40, max_depth=3, 
                                                  min_child_weight=10, gamma=0.5, subsample=0.6, 
                                                  colsample_bytree=0.75, reg_alpha=0.1, objective= 'binary:logistic', 
                                                  nthread=4,scale_pos_weight=1, seed=27), 
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'n_estimators': 180}, 0.7860847288742977)

与1）结果相比, 更差。所以, 仍选用学习速率较大时的结果。继续从1）循环调整, 此处时间原因不再调。

In [24]:
xgb = XGBClassifier(learning_rate =0.01, n_estimators=40, max_depth=3, min_child_weight=10, 
                    gamma=0.5, subsample=0.6, colsample_bytree=0.75, reg_alpha=0.1, 
                    objective= 'binary:logistic', nthread=4,scale_pos_weight=1, seed=27)
xgb.fit(X_train, y_train)
model_metrics(xgb, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.7953 测试集： 0.7884
[auc值] 训练集： 0.7995 测试集： 0.7780


## LightGBM模型

> 类似XGB， 先调整boosting框架参数, 再调整个体学习器参数。

In [25]:
# 首先看一下默认参数的结果
lgb0 = LGBMClassifier()
lgb0.fit(X_train, y_train)

model_metrics(lgb0, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.9943 测试集： 0.7968
[auc值] 训练集： 1.0000 测试集： 0.7829


In [106]:
# boosting框架参数
param_test = {'n_estimators':range(20,200,20)}
gsearch = GridSearchCV(estimator = LGBMClassifier(learning_rate =0.1, n_estimators=140,
                                                  max_depth=5,min_child_weight=1, gamma=0, 
                                                  subsample=0.8, colsample_bytree=0.8,seed=27),
                        param_grid = param_test, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

...略

# 遇到的问题

> XGB调参完还没有默认参数好, 一方面原因是因为没细调，另外是有5折时对score取平均的原因吗？

# Reference

[XGB调参](https://blog.csdn.net/han_xiaoyang/article/details/52665396)

[XGBoost和LightGBM的参数以及调参](https://www.jianshu.com/p/1100e333fcab)